
As informações foram extraídas dos 3 sites de checagem de fatos mais populares do Brasil. São eles:

1. To the Facts: https://www.aosfatos.org/
2. Public Agency: https://piaui.folha.uol.com.br/lupa/
3. Lupa Agency: https://apublica.org/

In [ ]:
import pandas as pd
import numpy as np
import re # regex
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
'''
Stopwords são palavras que são filtradas antes ou após o processamento de um texto.
Quando estamos trabalhando com análise de texto ou processamento de linguagem natural,
essas palavras podem adicionar muito ruído. Isso ocorre porque elas são as palavras
mais comuns em um idioma (por exemplo, "o", "a", "é", "de" em português) e aparecem
com frequência em vários contextos.

Essas palavras são geralmente removidas porque não contribuem com informações
significativas para o conteúdo geral do texto. Por exemplo, ao construir um modelo
de tópicos, as stopwords podem levar a resultados enganosos, pois elas podem aparecer
com muita frequência, mas não necessariamente indicam um tópico específico.

A biblioteca NLTK em Python fornece uma lista de stopwords e vamos baixar e imprimir
as stopwords em português usando o seguinte código:
'''

nltk.download('stopwords')
print(stopwords.words('portuguese'))

['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvésse

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Abrindo o csv
df = pd.read_csv('/content/election_brazil_2018.csv', sep =';')
df.shape

(460, 4)

In [ ]:
df.head(10)

,id,texto,rotulo,fonte
0,1,"Salário Mínimo: R$ 950,00. Bolsa Presidiário: ...",FALSO,AOS FATOS
1,2,Empresa contratada pelo TSE para apuração dos ...,FALSO,AOS FATOS
2,3,"O Aloizio Mercadante, ministro da Educação, mo...",FALSO,AOS FATOS
3,4,Há um complô espalhando fake news descaradas e...,FALSO,AOS FATOS
4,5,"Somente em 2017, mais de 800 milhões de tonela...",VERDADE,AOS FATOS
5,6,Nunca vi o Lula pronunciar essa palavra fascis...,FALSO,AOS FATOS
6,7,"O Mourão, por exemplo, foi ele próprio tortura...",FALSO,AOS FATOS
7,8,"O PSB, todos os seus governadores e o seu pres...",FALSO,AOS FATOS
8,9,Bolsonaro Nunca aprovou um projeto de seguranç...,VERDADE,AOS FATOS
9,10,Ele Lula não pode aparecer mais que 25% no hor...,VERDADE,AOS FATOS


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460 entries, 0 to 459
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      460 non-null    int64 
 1   texto   460 non-null    object
 2   rotulo  460 non-null    object
 3   fonte   460 non-null    object
dtypes: int64(1), object(3)
memory usage: 14.5+ KB


In [ ]:
df.isnull().sum()

id        0
texto     0
rotulo    0
fonte     0
dtype: int64

In [ ]:
'''

A função de “stemming” é um processo usado em processamento de linguagem natural para
reduzir palavras inflexionadas (ou por vezes derivadas) ao seu tronco, base ou raiz,
geralmente uma forma da palavra escrita.

Por exemplo, se aplicarmos o stemming às palavras “correndo”, “corredor”, “correu”,
todas elas seriam reduzidas para a palavra raiz “corr”.

O objetivo do stemming é reduzir a dimensionalidade do nosso conjunto de dados e
agrupar palavras de mesma origem. Isso é útil em várias aplicações, como pesquisa de
texto, recuperação de informações e análise de sentimentos, onde queremos considerar
todas as formas diferentes de uma palavra como a mesma entidade.

'''

porter_stemmer = PorterStemmer()

def steming_tokenizer(df):
  words = re.sub(r'[^A-Za-z]', ' ', df).lower().split()
  words = [porter_stemmer.stem(word) for word in words if not word in stopwords.words('portuguese')]
  words = ' '.join(words)
  return words

In [ ]:
df['features'] = df['texto'].apply(steming_tokenizer)

In [ ]:
# imprimindo a raiz das palavras
print(df['features'])

0      sal rio m nimo r bolsa presidi rio r convenc p...
1              empresa contratada tse apura voto liga pt
2      aloizio mercadant ministro educa mobiliz estad...
3      h compl espalhando fake news descarada descont...
4      soment milh es tonelada carga movimentada port...
                             ...                        
455    criminoso tentou matar bolsonaro assessora cam...
456    n s ex rcito brasileiro perdemo tr s joven gar...
457    n consegu produzir prego brasil colocar forma ...
458    ltima elei es prefeito datafolha ltimo errou t...
459    senhor henriqu meirel president banco central ...
Name: features, Length: 460, dtype: object


In [ ]:
# separar em features e rótulos
X = df['features']
y = df['rotulo']

In [ ]:
X

0      sal rio m nimo r bolsa presidi rio r convenc p...
1              empresa contratada tse apura voto liga pt
2      aloizio mercadant ministro educa mobiliz estad...
3      h compl espalhando fake news descarada descont...
4      soment milh es tonelada carga movimentada port...
                             ...                        
455    criminoso tentou matar bolsonaro assessora cam...
456    n s ex rcito brasileiro perdemo tr s joven gar...
457    n consegu produzir prego brasil colocar forma ...
458    ltima elei es prefeito datafolha ltimo errou t...
459    senhor henriqu meirel president banco central ...
Name: features, Length: 460, dtype: object

In [ ]:
y

0        FALSO
1        FALSO
2        FALSO
3        FALSO
4      VERDADE
        ...   
455      FALSO
456    VERDADE
457      FALSO
458      FALSO
459    VERDADE
Name: rotulo, Length: 460, dtype: object

In [ ]:
# Convertendo texto em números, pois o modelo não entendo palavras, mas ele compreende,
# nesse caso, que se uma palavra for muito frequente em um texto, isso pode indicar que
# se trata de fake-news.

vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.fit_transform(X)

### Separando em treinamento e teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# treinando o modelo
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()

In [ ]:
# previsão
pred = lr.predict(X_test)
acc = accuracy_score(y_test, pred)

f'Acurácia: {acc:.2f}'

'Acurácia: 0.60'

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import PorterStemmer
from statistics import mean
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')

def clean_text(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]','', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+','', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[~`]', '', text)
    return text

porter_stemmer = PorterStemmer()

def stem_words(text):
  words = re.sub(r'[^A-Za-z]', ' ', text).lower().split()
  words = [porter_stemmer.stem(word) for word in words if not word in stopwords.words('portuguese')]
  words = ' '.join(words)
  return words

def extrair_informacoes(texto):
    total_palavras = len(texto.split())
    frases = re.split(r'[.!?]', texto)
    comprimento_medio_frases = mean(len(frase.split()) for frase in frases if frase.strip())
    palavras_unicas = set(texto.split())
    num_palavras_unicas = len(palavras_unicas)
    comprimento_palavras = [len(palavra) for palavra in texto.split()]
    comprimento_medio_palavras = mean(comprimento_palavras)
    return {
        'Total_Palavras': total_palavras,
        'Comprimento_Medio_Frases': comprimento_medio_frases,
        'Num_Palavras_Unicas': num_palavras_unicas,
        'Comprimento_Medio_Palavras': comprimento_medio_palavras
    }

dados = ["Empresa contratada pelo TSE para apuração dos votos tem ligação com o PT"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(dados)

dados_limpos = [clean_text(texto) for texto in dados]
stop = stopwords.words('portuguese')
dados_sem_stopwords = [' '.join([word for word in texto.split() if word not in stop]) for texto in dados_limpos]

dados_stemmed = [stem_words(texto) for texto in dados_sem_stopwords]
dados_informacoes = [extrair_informacoes(texto) for texto in dados_stemmed]

print(dados_informacoes)

[{'Total_Palavras': 7, 'Comprimento_Medio_Frases': 7, 'Num_Palavras_Unicas': 7, 'Comprimento_Medio_Palavras': 5}]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
